This week we are tasked with exploring Linear and Logistic Regression models to predict an outcome. Based on our initial Exploratory analysis, we will perform feature extraction and analysis to determine what relevant features can be used to build our model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [83]:
#Importing data
openpolicing_path="/Users/sandeep/OneDrive - Northeastern University/Fundamentals of AI/Group Project/ma_statewide_2020_04_01.csv"

data=pd.read_csv(openpolicing_path,low_memory=False)

In [84]:
data.head()

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,contraband_weapons,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
1,2,2007-06-07,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,...,False,False,False,False,True,other,NaN,Commercial,MA,White
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White
4,5,2007-06-07,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic


In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
raw_row_number                int64
date                          object
location                      object
county_name                   object
subject_age                   float64
subject_race                  object
subject_sex                   object
type                          object
arrest_made                   object
citation_issued               object
warning_issued                object
outcome                       object
contraband_found              object
contraband_drugs              object
contraband_weapons            object
contraband_alcohol            bool
contraband_other              object
frisk_performed               object
search_conducted              bool
search_basis                  object
reason_for_stop               object
vehicle_type                  object
vehicle_registration_state    object
raw_Race                      object
dty

In [86]:
##Convert date to datetype
import datetime
# data.date
#data["date"] = pd.to_datetime(data.date, format="%Y-%m-%d %H:%M")
data['date'] = data['date'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d %I:%M %p'))
#data['date']=[datetime.datetime.strptime(d, "%Y-%m-%d") for d in data["date"]] 
# data['date'] = pd.to_datetime(data['date']).dt.date

# data['time'] = pd.to_datetime(data['date']).dt.time

In [87]:
data

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,contraband_weapons,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race
0,1,2007-06-06 12:00 AM,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
1,2,2007-06-07 12:00 AM,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,...,False,False,False,False,True,other,NaN,Commercial,MA,White
2,3,2007-06-07 12:00 AM,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3,4,2007-06-07 12:00 AM,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White
4,5,2007-06-07 12:00 AM,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416233,3418294,2015-12-22 12:00 AM,DARTMOUTH,Bristol County,25.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
3416234,3418295,2015-12-27 12:00 AM,FALL RIVER,Bristol County,52.0,asian/pacific islander,female,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,Asian or Pacific Islander
3416235,3418296,2015-01-06 12:00 AM,NORTHAMPTON,Hampshire County,41.0,white,female,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3416236,3418297,2015-01-06 12:00 AM,HOLYOKE,Hampden County,26.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White


In [88]:
#Convering race, sex, location etc to categorical
data.iloc[:,np.r_[2:4,5:8,9:12,19:23]]=data.iloc[:,np.r_[2:4,5:8,9:12,19:23]].astype("category")

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
raw_row_number                int64
date                          object
location                      category
county_name                   category
subject_age                   float64
subject_race                  category
subject_sex                   category
type                          category
arrest_made                   object
citation_issued               category
warning_issued                category
outcome                       category
contraband_found              object
contraband_drugs              object
contraband_weapons            object
contraband_alcohol            bool
contraband_other              object
frisk_performed               object
search_conducted              bool
search_basis                  category
reason_for_stop               category
vehicle_type                  category
vehicle_registration_state    category
raw_Race        

In [90]:
#Checking proportion of missing values
data.isnull().sum()*100/len(data)

raw_row_number                 0.000000
date                           0.000000
location                       0.195127
county_name                    0.195127
subject_age                    4.625146
subject_race                   0.048709
subject_sex                    0.457316
type                           0.000000
arrest_made                    0.026813
citation_issued                0.026813
warning_issued                 0.026813
outcome                        0.200747
contraband_found              98.368673
contraband_drugs              98.368673
contraband_weapons            98.368673
contraband_alcohol             0.000000
contraband_other              98.368673
frisk_performed               98.400843
search_conducted               0.000000
search_basis                  98.506281
reason_for_stop               48.579432
vehicle_type                   0.145277
vehicle_registration_state     0.287275
raw_Race                       0.048709
dtype: float64

<!-- We will use the KNN Imputer for imputing values into the Age and Sex variables for better precision -->

We impute null values using an MICE imputer from fancyimpute, a popular algorithm.
Check imputation algorithms here:
https://www.kaggle.com/residentmario/simple-techniques-for-missing-data-imputation

# Please fix below code(s) to impute values for age and sex, the missing values are < 1% for sex so we can remove them for more precision for age it is 4.6%

In [91]:
#Removing null, and unknown values from race
data=data[-data["subject_race"].isin(["other","unknown",np.nan])]
data["subject_race"].unique()

[white, hispanic, black, asian/pacific islander]
Categories (4, object): [white, hispanic, black, asian/pacific islander]

In [92]:
df = pd.DataFrame(data)

Encoding categorical variables for future analyses

In [127]:
#Convert the object type variables to string
df['subject_sex'] = df['subject_sex'].astype(str)
df['subject_race'] = df['subject_race'].astype(str)
df['type'] = df['type'].astype(str)
df['arrest_made'] = df['arrest_made'].astype(str)
df['citation_issued'] = df['citation_issued'].astype(str)
df['outcome'] = df['outcome'].astype(str)
df['contraband_found'] = df['contraband_found'].astype(str)
df['contraband_drugs'] = df['contraband_drugs'].astype(str)
df['warning_issued'] = df['warning_issued'].astype(str)
df['contraband_weapons'] = df['contraband_weapons'].astype(str)
df['contraband_alcohol'] = df['contraband_alcohol'].astype(str)
df['contraband_other'] = df['contraband_other'].astype(str)
df['frisk_performed'] = df['frisk_performed'].astype(str)
df['search_conducted'] = df['search_conducted'].astype(str)
df['search_basis'] = df['search_basis'].astype(str)
df['reason_for_stop'] = df['reason_for_stop'].astype(str)
df['vehicle_type'] = df['vehicle_type'].astype(str)
df['vehicle_registration_state'] = df['vehicle_registration_state'].astype(str)
df['raw_Race'] = df['raw_Race'].astype(str)

In [128]:
# One hot encoding race and other categorical variables
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['subject_race'] = label_encoder.fit_transform(df['subject_race'])
df['arrest_made'] = label_encoder.fit_transform(df['arrest_made'])
df['citation_issued'] = label_encoder.fit_transform(df['citation_issued'])
df['outcome'] = label_encoder.fit_transform(df['outcome'])
df['contraband_found'] = label_encoder.fit_transform(df['contraband_found'])
df['contraband_drugs'] = label_encoder.fit_transform(df['contraband_drugs'])
df['contraband_weapons'] = label_encoder.fit_transform(df['contraband_weapons'])
df['contraband_alcohol'] = label_encoder.fit_transform(df['contraband_alcohol'])
df['contraband_other'] = label_encoder.fit_transform(df['contraband_other'])

In [ ]:
!pip install suntime

In [ ]:
from suntime import Sun, SunTimeException

latitude = 42.407211
longitude = -71.382439

sun = Sun(latitude, longitude)

# Get sunrise and sunset in UTC
sunrise=[]
sunset=[]
for value in data["date"]:
    sunrise.append(sun.get_sunrise_time(value))
    sunset.append(sun.get_sunset_time(value))
    format(value.strftime('%H:%M'))
                  
data["sunrise"]=sunrise
data["sunset"]=sunset
print(data)

We now create new values for search, success rates

In [93]:
group_agg = data.groupby(["date"]).agg({
  "search_conducted" : lambda x: x.mean(skipna = True), 
  "frisk_performed" : lambda x: x.mean(skipna = True), 
  "contraband_found" : lambda x: x.mean(skipna = True)
  })

In [94]:
group_agg.rename(columns={"search_conducted" : 'search_rate', "frisk_performed" : 'frisk_rate',"contraband_found":'success_rate'}, inplace=True)

print(group_agg)

                     search_rate  frisk_rate  success_rate
date                                                      
2007-01-01 12:00 AM     0.039474         0.5      0.333333
2007-01-02 12:00 AM     0.021739         0.0      1.000000
2007-01-03 12:00 AM     0.000000         NaN           NaN
2007-01-04 12:00 AM     0.000000         NaN           NaN
2007-01-05 12:00 AM     0.024390         0.0      0.000000
...                          ...         ...           ...
2015-12-27 12:00 AM     0.009747         0.4      0.600000
2015-12-28 12:00 AM     0.008997         0.0      0.571429
2015-12-29 12:00 AM     0.008380         0.0      0.666667
2015-12-30 12:00 AM     0.003106         0.0      0.000000
2015-12-31 12:00 AM     0.008902         0.0      1.000000

[3287 rows x 3 columns]


In [95]:
group_df = pd.DataFrame()
group_df = group_agg['search_rate']
group_df.iloc[1:]

date
2007-01-02 12:00 AM    0.021739
2007-01-03 12:00 AM    0.000000
2007-01-04 12:00 AM    0.000000
2007-01-05 12:00 AM    0.024390
2007-01-06 12:00 AM    0.000000
                         ...   
2015-12-27 12:00 AM    0.009747
2015-12-28 12:00 AM    0.008997
2015-12-29 12:00 AM    0.008380
2015-12-30 12:00 AM    0.003106
2015-12-31 12:00 AM    0.008902
Name: search_rate, Length: 3286, dtype: float64

In [103]:
median = df['subject_age'].median()
df['subject_age'].fillna(median, inplace=True)

In [104]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, train_size=0.7, test_size=0.3, shuffle=True, random_state=100)

In [105]:
X_train = train_data['date']
y_train = train_data['subject_age']
X_train = X_train.values.reshape(-1, 1)
y_train = y_train.values.reshape(-1, 1)

In [114]:
X_test = test_data['date']
y_test = test_data['subject_age']
X_test = X_test.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [115]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)
lm_pred = lm.predict(X_test)

In [124]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, mean_squared_error, mean_absolute_error, explained_variance_score

mse = mean_squared_error(y_test, lm_pred)
mae = mean_absolute_error(y_test, lm_pred)
evs = explained_variance_score(y_test, lm_pred)

metrics = {"mse":mse,
           "mae":mae,
           "evs":evs
}

In [126]:
metrics

{'mse': 175.22525230474423,
 'mae': 10.937961367039547,
 'evs': 0.002908649049812473}

In [112]:
import statsmodels.api as sm

X_train_new = sm.add_constant(X_train)
lm_1 = sm.OLS(y_train, X_train).fit()
print(lm_1.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.883
Model:                            OLS   Adj. R-squared (uncentered):              0.883
Method:                 Least Squares   F-statistic:                          1.783e+07
Date:                Fri, 06 Nov 2020   Prob (F-statistic):                        0.00
Time:                        23:35:17   Log-Likelihood:                     -9.4916e+06
No. Observations:             2370584   AIC:                                  1.898e+07
Df Residuals:                 2370583   BIC:                                  1.898e+07
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------